In [2]:
import getpass
import os
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
import openai 
# from langgraph_supervisor import create_supervisor
from langchain.chat_models import init_chat_model
from textwrap import dedent

In [3]:
# Prompts
TARGET_DOMAIN = dedent("""
        As a Domain Analysis Specialist, extract the core innovation domain from the user query.
        Instructions:
        1. Analyze the user's input
        2. Identify the primary domain requiring innovation
        3. Classify it within standard innovation categories
        Output Format:
        Target Domain: [Clear, specific domain label]
        Be very detailed and specific in your response and do not generalize. Respond ONLY with the name of the domain, do NOT include ANY other text like 'Target Domain:'.
        User query: {user_query}
""").strip()

PROBLEM_LANDSCAPE = dedent("""
        You are a Problem Landscape Analyst. Your task is to map out the concrete challenges within the target domain identified.
        Instructions:
        1. Identify 3-5 core problems or challenges currently present in this domain.
        2. For each problem, provide:
        - Problem: A short, clear title.
        - Description: 2-3 sentences explaining what the problem is and why it matters.
        - Context: Briefly state the circumstances or environment where this problem occurs.
        - Stakeholders: List the main groups or individuals affected.
        - Root Causes: Identify 1-3 underlying causes, if known.
        - Impact: State the significance of the problem (e.g., social, economic, technical).
        - Current Approaches: How is this problem currently addressed?
        - Limitations: What are the shortcomings of current approaches?
        - Success Metrics: How would you measure if this problem is solved?
        - Interconnections: Note if this problem is linked to or influenced by other problems.
        Output Format:
        Present your findings as a structured list or JSON array, with each problem fully described as above.
        Important:
        - Focus on clarity and completeness.
        - Avoid abstracting or generalizing; stay concrete and domain-specific.
        - Do not propose solutions; only describe the current problem landscape.
        Target domain: {target_domain}
""").strip()

ABSTRACTION = dedent("""
You are a TRIZ Methodology Expert. Transform domain-specific problems into universal contradictions.
        Process:
        1. For each problem in the problem landscape
        - Abstract to universal parameters (what improves vs. what worsens)
        - Express as 'When we improve X, Y worsens'
        - Ensure parameters are domain-agnostic
        2. Identify 3-4 core contradictions:
        - Select the most fundamental tensions
        - Map to TRIZ contradiction matrix
        - Note applicable inventive principles
        Output:
        # Core Contradictions:
        1. Improving [parameter] vs. Worsening [parameter]
        - TRIZ Principles: [1-3 relevant principles]
        - Innovation Potential: [High/Medium/Low]
        - Universal Application: [Brief example from another domain]
        Focus on contradictions that, if resolved, would create breakthrough value.

        Problem landscape: {problem_landscape}
""").strip()

BASE_DOMAIN = dedent("""
        You are a Cross-Domain Search Specialist.
        For each contradiction provided, identify two distinct source domains (fields or industries) where this contradiction has been successfully addressed.
        The domains should have A CONCEPTUAL DISTANCE OF AT LEAST 2 DISTINCT HOPS FROM WHAT IMMEDIATELY COMES TO MIND. Be creative! It can be domains within spheres like natural, phsyical, social, artistic, anything.
        For each domain, briefly explain why it is relevant to the contradiction. Do not describe specific solutions-only list the domains and your rationale.
        Output:
        A list for each contradiction, naming two relevant domains with a one-sentence rationale for each.

        Contradictions: {contradictions}
""").strip()

BASE_SOLUTIONS = dedent("""
        You are a Solution Pattern Extractor. You are provided with an input with 2 base domains identified per contradiction.
        For each of these identified base domains , identify one specific, well-documented solution pattern within the domain that effectively resolves the contradiction.
        For each solution pattern:
        - The original base domain identified
        - The name or label of the solution pattern
        - A detailed description of the core mechanism or principle involved and how it addressed the domain's contradiction
        - The context or situation in the domain where this pattern is applied\n"
        Do not generalize or adapt the solution-simply describe how the contradiction is addressed within each source domain.
        Output:
        For each of the provided domain, list the base domain name, solution pattern name, a detailed description of its mechanism, and the context in which it is used.

        Input: {input}
""").strip()

ANALOGICAL_TRANSFER = dedent("""
You are an Analogical Transfer Specialist.

Your task is to propose how solution patterns used to resolve abstracted contradictions in various base domains might inspire solution framings for the original target domain.

Input:
1. A list of abstracted contradictions, each with two base domains and their associated solution patterns (including how each contradiction was resolved in those domains).
2. The original target domain.

Instructions:
For each contradiction, review the solution patterns from both base domains. For each pattern:
- Analyze the core mechanism or principle behind the solution.
- Map and adapt this mechanism conceptually to the target domain, considering the specific context and needs of the target domain.
- Clearly describe how this analogical transfer could frame a potential solution in the target domain.
- Highlight any key adaptations, considerations, or limitations that would be relevant when applying this pattern to the target domain.

Output:
For each contradiction and base domain solution pattern, provide a comprehensive description of a proposed solution framing for the target domain, including:
- The original contradiction addressed
- The source domain and solution pattern
- A detailed explanation of how the pattern could inspire or inform a solution in the target domain
- Any important adaptations or considerations for successful transfer

Input:
- List of abstracted contradictions: {contradictions_solutions}
- Original target domain: {target_domain}
""").strip()

In [6]:
# check for openai API
def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")


_set_if_undefined("OPENAI_API_KEY")

In [7]:
# Graph state: workflow
class ReasoningState(TypedDict):
    user_query: str
    target_domain: str
    problem_landscape: str
    abstraction: str
    base_domain: str
    base_solutions: str
    analogical_transfer: str
    solution: str

In [8]:
# Can switch between different LLMs 
llms = init_chat_model("openai:gpt-4.1")

In [9]:
# 1st Agent: Identify Target Domain from User Input
def target_domain_agent(state: ReasoningState):
    u = state['user_query']

    msg = llms.invoke(TARGET_DOMAIN.format(user_query=u))

    return {"target_domain": msg.content}

In [10]:
# 2st Agent: Conduct comprehensive research into problem landscape for target domain 
# (i.e. What specific challenges exist in this domain?)
def problem_landscape_agent(state: ReasoningState):
    t = state['target_domain']

    msg = llms.invoke(PROBLEM_LANDSCAPE.format(target_domain=t))

    return {"problem_landscape": msg.content}

In [11]:
# 3rd Agent: Abstract Problems Identified into Generalized Principles and TRIZ Contradiction
def abstraction_agent(state: ReasoningState):
    p = state['problem_landscape']

    msg = llms.invoke(ABSTRACTION.format(problem_landscape=p))

    return {"abstraction": msg.content}

In [12]:
# 4th Agent: Search for Appropriate Base Domains
def base_domain_agent(state: ReasoningState):
    a = state['abstraction']

    msg = llms.invoke(BASE_DOMAIN.format(contradictions=a))

    return {"base_domain": msg.content}

In [13]:
# 5th Agent: Identify Solution in Base Domain
def base_solution_agent(state: ReasoningState):
    b = state['base_domain']

    msg = llms.invoke(BASE_SOLUTIONS.format(input=b))

    return {"base_solutions": msg.content}

In [14]:
# 6th Agent: Base Domain Solution Informing Target Domain Solution
def analogical_transfer_agent(state: ReasoningState):
    if "base_solutions" not in state:
        raise ValueError("Missing 'base_solutions' key. Check if previous node returned it.")
    b = state['base_solutions']
    t = state['target_domain']
    
    msg = llms.invoke(ANALOGICAL_TRANSFER.format(contradictions_solutions=b, target_domain = t))

    return {"analogical_transfer": msg.content}

In [18]:
# 7th Agent: Summarize everything and respond to the question
def synthesis_agent(state: ReasoningState):
    msg = llms.invoke(
        f"Evaluate the proposed analogical solutions. Find the best one that balances practicality with innovation. Then, provide a detailed, well-structured response that addresses all aspects of the query.\n\n"
        f"Problem: {state['user_query']}\n"
        f"Analogical Solutions: {state['analogical_transfer']}"
        f"In your output, remember to abstract away the analogy itself such that it is focused on responding to the user input."
    )
    return {"solution": msg.content}

In [19]:
# Construct the workflow 
workflow = StateGraph(ReasoningState)

workflow.add_node("target", target_domain_agent)
workflow.add_node("landscape", problem_landscape_agent)
workflow.add_node("abstract", abstraction_agent)
workflow.add_node("base", base_domain_agent)
workflow.add_node("base_soln", base_solution_agent)
workflow.add_node("analogy", analogical_transfer_agent)
workflow.add_node("synthesis", synthesis_agent)

workflow.set_entry_point("target")

# Define edges
workflow.add_edge("target", "landscape")
workflow.add_edge("landscape", "abstract")
workflow.add_edge("abstract", "base")
workflow.add_edge("base", "base_soln")
workflow.add_edge("base_soln", "analogy")
workflow.add_edge("analogy", "synthesis")

workflow.set_finish_point("synthesis")

graph = workflow.compile()

In [20]:
test_q = "Teachers introduced structured group projects using peer evaluations and rotating roles to boost collaboration in science classes. Despite these measures, anonymous student surveys revealed that individual quiz scores post-project dropped compared to solo assignments, signaling diluted accountability. How can educators redesign design a system to ensure measurable individual mastery while preserving the benefits of teamwork?"
input_state = {"user_query": test_q}

final_state = graph.invoke(input_state)

In [21]:
# parse_output
import re

def parse_solution(text):
    # Remove Markdown formatting like "**" and "\n"
    clean_text = text.replace("**", "").replace("\\n", "\n")

    # Split into sections by horizontal rules (---)
    sections = re.split(r'-{3,}', clean_text)

    # Create a readable version of each section
    readable_output = []
    for section in sections:
        section = section.strip()
        if not section:
            continue
        # Optional: separate title and body if present
        lines = section.split("\n")
        if len(lines) > 1 and ":" not in lines[0]:
            # First line is likely a heading
            heading = lines[0]
            body = "\n".join(lines[1:])
            readable_output.append(f"\n=== {heading} ===\n{body}")
        else:
            readable_output.append(section)

    return "\n\n".join(readable_output)

In [22]:
# Save all parsed outputs to a text file
with open("sample_output.txt", "w", encoding="utf-8") as f:
    for key in final_state:
        raw_output = str(final_state[key])  # Ensure it's a string
        final_output = parse_solution(raw_output)
        f.write(f"\n### {key} ###\n")
        f.write(final_output)
        f.write("\n\n")

In [35]:
# Generate output without analogical reasoning Pipeline for comparison
def basic_output(user_input: str):
    
    prompt = f"""
    {user_input}
    """

    response = llms.invoke(prompt) 

    # Parse and format the response
    formatted_response = parse_solution(response.content)

    return(formatted_response)

In [31]:
test_q = "Teachers introduced structured group projects using peer evaluations and rotating roles to boost collaboration in science classes. Despite these measures, anonymous student surveys revealed that individual quiz scores post-project dropped compared to solo assignments, signaling diluted accountability. How can educators redesign design a system to ensure measurable individual mastery while preserving the benefits of teamwork?"
result = basic_output(test_q)

print("\n=== GPT-4.1 Response Without Analogical Reasoning Pipeline ===\n")



=== GPT-4.1 Response Without Analogical Reasoning Pipeline ===



In [32]:
## GPT 4.1 without Analogical Reasoning Without CoT
print(result)

You’re facing a classic challenge: balancing authentic collaboration with the need for individual accountability and measurable learning outcomes. Below are several evidence-based strategies to ensure students maintain and demonstrate individual mastery, even within group-based science projects:


=== ### 1. Hybrid Assessment Model ===

- Combine group and individual grades:  
  Assign a portion of the project grade to the *group* (based on final product, teamwork, etc.), but *reserve a significant percentage (e.g., 40–60%)* for individual performance, demonstrated through:
    - Quizzes/tests on project content
    - Individual write-ups or reflections
    - “Exit tickets” or quick checks addressing key concepts


=== ### 2. Embedded Individual Tasks ===

- Interleaved Solo Work:  
  Require each student to submit or present a distinct section of the project—such as a data analysis, explanation, or conclusion—in addition to the group submission.
- Rotating “Expert” Check-ins:  
  Each

In [37]:
# Generate output without analogical reasoning Pipeline BUT WITH COT for comparison
def basic_output_COT(user_input: str):
    
    prompt = f"""
    {user_input}
    Think step by step.
    """

    response = llms.invoke(prompt) 

    # Parse and format the response
    formatted_response = parse_solution(response.content)

    return(formatted_response)

In [38]:
test_q = "Teachers introduced structured group projects using peer evaluations and rotating roles to boost collaboration in science classes. Despite these measures, anonymous student surveys revealed that individual quiz scores post-project dropped compared to solo assignments, signaling diluted accountability. How can educators redesign design a system to ensure measurable individual mastery while preserving the benefits of teamwork?"
result = basic_output_COT(test_q)

print("\n=== GPT-4.1 Response Without Analogical Reasoning Pipeline WITH COT ===\n")
print(result)


=== GPT-4.1 Response Without Analogical Reasoning Pipeline WITH COT ===

Certainly! To address the problem—group work fostering teamwork but possibly reducing individual mastery/accountability—educators should reconsider both the design and assessment of group projects. Here’s a step-by-step approach:

### Step 1: Diagnose the Core Issues
- Observation: Individual post-project quizzes show lower scores than after solo work.
- Analysis: Students may be "hitchhiking" (letting groupmates do more), or division of labor is masking some students’ conceptual gaps.

### Step 2: Set Clear Dual Objectives
- Maintain collaboration (teamwork, communication, problem-solving).
- Ensure individual understanding and accountability (content mastery).

### Step 3: Redesign Project Structure

A. Roles and Responsibilities
- Continue rotating roles but with defined individual deliverables for each phase.
- Example: For a lab report, each student writes a different section (method, results, analysis) tied

In [39]:
# Generate output without analogical reasoning Pipeline BUT WITH PROMPT ENGINEERING for comparison (ask it to balance innovativeness with practicality)
def basic_output_prompt_engin(user_input: str):
    
    prompt = f"""
    {user_input}
    Try to balance practicality with innovation.
    """

    response = llms.invoke(prompt) 

    # Parse and format the response
    formatted_response = parse_solution(response.content)

    return(formatted_response)

In [40]:
test_q = "Teachers introduced structured group projects using peer evaluations and rotating roles to boost collaboration in science classes. Despite these measures, anonymous student surveys revealed that individual quiz scores post-project dropped compared to solo assignments, signaling diluted accountability. How can educators redesign design a system to ensure measurable individual mastery while preserving the benefits of teamwork?"
result = basic_output_prompt_engin(test_q)

print("\n=== GPT-4.1 Response Without Analogical Reasoning Pipeline WITH Prompt Engineering ===\n")
print(result)


=== GPT-4.1 Response Without Analogical Reasoning Pipeline WITH Prompt Engineering ===

Certainly! Balancing individual accountability with authentic collaboration is a common challenge in group work. Here’s a multi-pronged, practical-yet-innovative redesign educators can consider:


=== 1. Hybrid Assessment Model ===

- Personal Assignments Within Group Projects: Structure group projects so that each student is responsible for a distinct sub-task, deliverable, or research component. For example, in a biology project on ecosystems, one might investigate plant species, another animal relationships, etc. Require each member to submit a short individual report, reflection, or data analysis alongside the collective assignment.
- Individual “Defense”: After project completion, each student briefly presents or answers oral/written questions about *their* contribution and the overall project. This spot-checks true comprehension and holds each accountable for both their AND the group's work.
